In [6]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

device = torch.device("cuda")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

state_dict = torch.load("BART.pth", map_location=device)  # or "cuda" if using GPU
model.load_state_dict(state_dict)

model.to(device)
model.eval()

C:\Users\MSI-Admin\AppData\Local\Temp\ipykernel_28396\4247982673.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("BART.pth", map_location=device

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

In [7]:
def evaluate(model, tokenizer, input_text, device):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)

    output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    return tokenizer.decode(outputs[0], skip_special_tokens=True, truncation=False)

In [9]:
user = ""

with open("chat_output.txt", "w") as file:
    while user != "-1":
        user = input("Enter command: ")
        if user != "-1":
            output = evaluate(model, tokenizer, user, device)
            print(f"User: {user}")
            print(f"Output: {output}")

            file.write(f"User: {user}\n")
            file.write(f"Output: {output}\n\n")

print("Chat Ended")

User: right 2
Output: X += 2, End Task
User: up 15
Output: Z += 15, End Task
User: left 6
Output: X += 6, End Task
User: move down by 16
Output: Z -= 16, End Task
User: move to the right by 18
Output: label, X += 18, End Task
User: move up by 19
Output: Z += 19, End Task
User: move forward by 16 and then to the right by 7
Output: Y += 16, Z += 7, End Task
User: down by 7
Output: Z -= 7, End Task
User: right by 7
Output: X += 7, End Task
Chat Ended


In [11]:
import pandas as pd

increments = []

with open("chat_output.txt", "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith("Output:"):
            output_line = line.replace("Output:", "").strip().replace("End Task", "")
            parts = output_line.split(",")

            for part in parts:
                part = part.strip()
                dx = dy = dz = 0
                if part.startswith("X +="):
                    dx = int(part.split("+=")[1].strip())
                elif part.startswith("X -="):
                    dx = -int(part.split("-=")[1].strip())
                elif part.startswith("Y +="):
                    dy = int(part.split("+=")[1].strip())
                elif part.startswith("Y -="):
                    dy = -int(part.split("-=")[1].strip())
                elif part.startswith("Z +="):
                    dz = int(part.split("+=")[1].strip())
                elif part.startswith("Z -="):
                    dz = -int(part.split("-=")[1].strip())
                else:
                    continue

                increments.append((dx, dy, dz))

df = pd.DataFrame(increments, columns=["X", "Y", "Z"])
df.to_csv("movement_increments.csv", index=False)

print("Saved to movement_increments.csv")

import subprocess
subprocess.run(["git", "add", "movement_increments.csv"])
subprocess.run(["git", "commit", "-m", "Update movement increments (separate lines)"])
subprocess.run(["git", "push"])

Saved to movement_increments.csv


CompletedProcess(args=['git', 'push'], returncode=0)